In [28]:
import pandas as pd
import os
import numpy as np
from keras.layers import Input, Embedding, Flatten, Dot, Add, Concatenate
from keras.layers import Dense, Dropout, BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.models import Model
from keras.callbacks import Callback

In [29]:
user_info = pd.read_csv('../data/interim/user_occupation_table.csv')
user_ids = user_info['user_id']
user_info.drop(['user_id', 'Unnamed: 0'], axis=1, inplace=True)

movie_info = pd.read_csv('../data/interim/movie_genre_table.csv')
movie_ids = movie_info['item_id']
movie_info.drop(['item_id', 'Unnamed: 0'], axis=1, inplace=True)


In [30]:
def define(lat_dims, num, info):
    emb_input = Input(shape=(1, ))
    emb = Embedding(num+1, lat_dims,
                   embeddings_initializer='random_normal', trainable=True)(emb_input)
    emb = Flatten()(emb)

    bias = Embedding(num+1, 1, embeddings_initializer='zeros', trainable=True)(emb_input)
    bias = Flatten()(bias)

    info_emb = Embedding(num, info.shape[1], weights=[info], trainable=False)(emb_input)
    info_emb = Flatten()(info_emb)

    return emb_input, emb, bias, info_emb

In [31]:
class MF():
    def __init__(self, lat_dims=16, num_users=943, num_movies=1682):
        self.lat_dims = lat_dims
        self.num_users = num_users
        self.num_movies = num_movies

    def __call__(self, user_info, movie_info):
        input_ = Input(shape=(1, ))
        u_input, user, user_bias, user_info = define(self.lat_dims, self.num_users, user_info)
        m_input, movie, movie_bias, movie_info = define(self.lat_dims, self.num_movies, movie_info)

        dot = Dot(axes=1)([user, movie])
        output = Add()([dot, user_bias, movie_bias])
        model = Model(inputs=[u_input, m_input], outputs=output)
        return model


In [32]:
class DNN():
    def __init__(self, dropout=0.5, lat_dims=64, num_users=943, num_movies=1682):
        self.lat_dims = lat_dims
        self.num_users = num_users
        self.num_movies = num_movies
        self.dropout = dropout
    
    def __call__(self, user_info, movie_info, info):
        u_input, user, user_bias, user_info = define(self.lat_dims, self.num_users, user_info)
        m_input, movie, movie_bias, movie_info = define(self.lat_dims, self.num_movies, movie_info)
        
        user = Dropout(self.dropout)(user)
        movie = Dropout(self.dropout)(movie)
        if info:
            x = Concatenate()([user, movie, user_info, movie_info])
        else:
            x = Concatenate()([user, movie])
        x = Dense(256, activation='relu')(x)
        x = BatchNormalization()(x)
        x = Dropout(self.dropout)(x)
        x = Dense(256, activation='relu')(x)
        x = BatchNormalization()(x)
        x = Dropout(self.dropout)(x)
        x = Dense(256, activation='relu')(x)
        x = BatchNormalization()(x)
        x = Dropout(self.dropout)(x)
        
        output = Dense(1, activation='relu')(x)
        model = Model(inputs=[u_input, m_input], outputs=output)
        return model

In [33]:
class History(Callback):
    def on_train_begin(self, logs=None):
        self.train_loss = []
        self.val_loss = []

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        self.train_loss.append(logs.get('loss'))
        self.val_loss.append(logs.get('val_loss'))

In [41]:
def train(model_mf, model_dnn_info, user_info, movie_info, user_train, movie_train, ratings_train, user_test, movie_test, ratings_test, model_name_mf, model_name_dnn, model_name_dnn_info):

    model_mf.compile(loss='mse', optimizer='adam')
    model_dnn_info.compile(loss='mse', optimizer='adam')

    model_mf.summary()
    model_dnn_info.summary()

    checkpoint_mf = ModelCheckpoint(model_name_mf, monitor='val_loss', verbose=0,
                                     save_best_only=True, mode='min')
    checkpoint_dnn_info = ModelCheckpoint(model_name_dnn_info, monitor='val_loss', verbose=0,
                                     save_best_only=True, mode='min')

    history_mf = History()
    callbacks_list_mf = [checkpoint_mf, history_mf]

    history_dnn_info = History()
    callbacks_list_dnn_info = [checkpoint_dnn_info, history_dnn_info]

    model_mf.fit([user_train, movie_train], ratings_train, batch_size=256, epochs=100, 
                 validation_data=([user_test, movie_test], ratings_test), callbacks=callbacks_list_mf)

    model_dnn_info.fit([user_train, movie_train], ratings_train, batch_size=256, epochs=100, 
                 validation_data=([user_test, movie_test], ratings_test), callbacks=callbacks_list_dnn_info)

In [37]:
# Training on all datasets separately
for i in range(7):
    X_train = pd.read_csv(f'../data/interim/X_train_{i+1}.csv')
    X_test = pd.read_csv(f'../data/interim/X_test_{i+1}.csv')

    user_train = X_train['user_id']
    movie_train = X_train['item_id']
    ratings_train = X_train['ratings']

    user_test = X_test['user_id']
    movie_test = X_test['item_id']
    ratings_test = X_test['ratings']

    model_dir = f'../models/train{i+1}'
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    model_name_mf = os.path.join(model_dir, "mf_model_e{epoch:02d}.hdf5")
    model_name_dnn = os.path.join(model_dir, "dnn_model_e{epoch:02d}.hdf5")
    model_name_dnn_info = os.path.join(model_dir, "dnn_info_model_e{epoch:02d}.hdf5")
    
    model_mf = MF()
    model_mf = model_mf(user_info, movie_info)
    model_dnn_info = DNN()
    model_dnn_info = model_dnn_info(user_info, movie_info, False)
    
    train(model_mf, model_dnn_info, user_info, movie_info, user_train, movie_train, ratings_train, user_test, movie_test, ratings_test, model_name_mf, model_name_dnn, model_name_dnn_info)

Model: "model_14"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_37 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 input_38 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 embedding_84 (Embedding)       (None, 1, 16)        15104       ['input_37[0][0]']               
                                                                                                  
 embedding_87 (Embedding)       (None, 1, 16)        26928       ['input_38[0][0]']               
                                                                                           

Epoch 3/100
313/313 [==============================] - 1s 3ms/step - loss: 0.7921 - val_loss: 0.7852
Epoch 4/100
313/313 [==============================] - 1s 3ms/step - loss: 0.7176 - val_loss: 0.7364
Epoch 5/100
313/313 [==============================] - 1s 3ms/step - loss: 0.6533 - val_loss: 0.7044
Epoch 6/100
313/313 [==============================] - 1s 4ms/step - loss: 0.6026 - val_loss: 0.6859
Epoch 7/100
313/313 [==============================] - 1s 3ms/step - loss: 0.5627 - val_loss: 0.6762
Epoch 8/100
313/313 [==============================] - 1s 4ms/step - loss: 0.5306 - val_loss: 0.6725
Epoch 9/100
313/313 [==============================] - 1s 3ms/step - loss: 0.5040 - val_loss: 0.6725
Epoch 10/100
313/313 [==============================] - 1s 3ms/step - loss: 0.4813 - val_loss: 0.6749
Epoch 11/100
313/313 [==============================] - 1s 3ms/step - loss: 0.4618 - val_loss: 0.6793
Epoch 12/100
313/313 [==============================] - 1s 3ms/step - loss: 0.4449 - val_

Epoch 63/100
313/313 [==============================] - 3s 11ms/step - loss: 1.0000 - val_loss: 1.0000
Epoch 64/100
313/313 [==============================] - 4s 11ms/step - loss: 1.0000 - val_loss: 1.0000
Epoch 65/100
313/313 [==============================] - 4s 11ms/step - loss: 1.0000 - val_loss: 1.0000
Epoch 66/100
313/313 [==============================] - 5s 17ms/step - loss: 1.0000 - val_loss: 1.0000
Epoch 67/100
313/313 [==============================] - 4s 14ms/step - loss: 1.0000 - val_loss: 1.0000
Epoch 68/100
313/313 [==============================] - 4s 12ms/step - loss: 1.0000 - val_loss: 1.0000
Epoch 69/100
313/313 [==============================] - 4s 12ms/step - loss: 1.0000 - val_loss: 1.0000
Epoch 70/100
313/313 [==============================] - 4s 11ms/step - loss: 1.0000 - val_loss: 1.0000
Epoch 71/100
313/313 [==============================] - 4s 11ms/step - loss: 1.0000 - val_loss: 1.0000
Epoch 72/100
313/313 [==============================] - 4s 11ms/step - lo

 flatten_102 (Flatten)          (None, 64)           0           ['embedding_102[0][0]']          
                                                                                                  
 flatten_105 (Flatten)          (None, 64)           0           ['embedding_105[0][0]']          
                                                                                                  
 dropout_40 (Dropout)           (None, 64)           0           ['flatten_102[0][0]']            
                                                                                                  
 dropout_41 (Dropout)           (None, 64)           0           ['flatten_105[0][0]']            
                                                                                                  
 concatenate_8 (Concatenate)    (None, 128)          0           ['dropout_40[0][0]',             
                                                                  'dropout_41[0][0]']             
          

313/313 [==============================] - 4s 12ms/step - loss: 0.8734 - val_loss: 0.8838
Epoch 26/100
313/313 [==============================] - 4s 12ms/step - loss: 0.8720 - val_loss: 0.8839
Epoch 27/100
313/313 [==============================] - 4s 12ms/step - loss: 0.8720 - val_loss: 0.8844
Epoch 28/100
313/313 [==============================] - 4s 12ms/step - loss: 0.8684 - val_loss: 0.8848
Epoch 29/100
313/313 [==============================] - 4s 12ms/step - loss: 0.8680 - val_loss: 0.8873
Epoch 30/100
313/313 [==============================] - 4s 12ms/step - loss: 0.8675 - val_loss: 0.8837
Epoch 31/100
313/313 [==============================] - 4s 12ms/step - loss: 0.8659 - val_loss: 0.8834
Epoch 32/100
313/313 [==============================] - 4s 12ms/step - loss: 0.8658 - val_loss: 0.8869
Epoch 33/100
313/313 [==============================] - 4s 13ms/step - loss: 0.8657 - val_loss: 0.8806
Epoch 34/100
313/313 [==============================] - 4s 13ms/step - loss: 0.8647 - 

...

 concatenate_12 (Concatenate)   (None, 128)          0           ['dropout_60[0][0]',             
                                                                  'dropout_61[0][0]']             
                                                                                                  
 dense_48 (Dense)               (None, 256)          33024       ['concatenate_12[0][0]']         
                                                                                                  
 batch_normalization_36 (BatchN  (None, 256)         1024        ['dense_48[0][0]']               
 ormalization)                                                                                    
                                                                                                  
 dropout_62 (Dropout)           (None, 256)          0           ['batch_normalization_36[0][0]'] 
                                                                                                  
 dense_49 

354/354 [==============================] - 4s 12ms/step - loss: 1.0000 - val_loss: 1.0000
Epoch 34/100
354/354 [==============================] - 4s 12ms/step - loss: 1.0000 - val_loss: 1.0000
Epoch 35/100
354/354 [==============================] - 4s 12ms/step - loss: 1.0000 - val_loss: 1.0000
Epoch 36/100
354/354 [==============================] - 4s 12ms/step - loss: 1.0000 - val_loss: 1.0000
Epoch 37/100
354/354 [==============================] - 4s 12ms/step - loss: 1.0000 - val_loss: 1.0000
Epoch 38/100
354/354 [==============================] - 4s 12ms/step - loss: 1.0000 - val_loss: 1.0000
Epoch 39/100
354/354 [==============================] - 4s 12ms/step - loss: 1.0000 - val_loss: 1.0000
Epoch 40/100
354/354 [==============================] - 4s 12ms/step - loss: 1.0000 - val_loss: 1.0000
Epoch 41/100
354/354 [==============================] - 4s 12ms/step - loss: 1.0000 - val_loss: 1.0000
Epoch 42/100
354/354 [==============================] - 4s 12ms/step - loss: 1.0000 - 

                                                                                                  
 flatten_159 (Flatten)          (None, 16)           0           ['embedding_159[0][0]']          
                                                                                                  
 embedding_157 (Embedding)      (None, 1, 1)         944         ['input_67[0][0]']               
                                                                                                  
 embedding_160 (Embedding)      (None, 1, 1)         1683        ['input_68[0][0]']               
                                                                                                  
 dot_13 (Dot)                   (None, 1)            0           ['flatten_156[0][0]',            
                                                                  'flatten_159[0][0]']            
                                                                                                  
 flatten_1

Epoch 15/100
354/354 [==============================] - 1s 3ms/step - loss: 0.4069 - val_loss: 0.7718
Epoch 16/100
354/354 [==============================] - 1s 3ms/step - loss: 0.3980 - val_loss: 0.7799
Epoch 17/100
354/354 [==============================] - 1s 3ms/step - loss: 0.3901 - val_loss: 0.7880
Epoch 18/100
354/354 [==============================] - 1s 3ms/step - loss: 0.3831 - val_loss: 0.7959
Epoch 19/100
354/354 [==============================] - 1s 3ms/step - loss: 0.3767 - val_loss: 0.8040
Epoch 20/100
354/354 [==============================] - 1s 3ms/step - loss: 0.3709 - val_loss: 0.8120
Epoch 21/100
354/354 [==============================] - 1s 3ms/step - loss: 0.3658 - val_loss: 0.8196
Epoch 22/100
354/354 [==============================] - 1s 4ms/step - loss: 0.3611 - val_loss: 0.8271
Epoch 23/100
354/354 [==============================] - 1s 3ms/step - loss: 0.3568 - val_loss: 0.8348
Epoch 24/100
354/354 [==============================] - 1s 4ms/step - loss: 0.3528

Epoch 75/100
354/354 [==============================] - 4s 12ms/step - loss: 1.0000 - val_loss: 1.0000
Epoch 76/100
354/354 [==============================] - 4s 12ms/step - loss: 1.0000 - val_loss: 1.0000
Epoch 77/100
354/354 [==============================] - 4s 12ms/step - loss: 1.0000 - val_loss: 1.0000
Epoch 78/100
354/354 [==============================] - 4s 12ms/step - loss: 1.0000 - val_loss: 1.0000
Epoch 79/100
354/354 [==============================] - 4s 12ms/step - loss: 1.0000 - val_loss: 1.0000
Epoch 80/100
354/354 [==============================] - 4s 12ms/step - loss: 1.0000 - val_loss: 1.0000
Epoch 81/100
354/354 [==============================] - 4s 13ms/step - loss: 1.0000 - val_loss: 1.0000
Epoch 82/100
354/354 [==============================] - 4s 12ms/step - loss: 1.0000 - val_loss: 1.0000
Epoch 83/100
354/354 [==============================] - 4s 12ms/step - loss: 1.0000 - val_loss: 1.0000
Epoch 84/100
354/354 [==============================] - 4s 12ms/step - lo

In [43]:
# Training on all data sets in turn (while maintaining weights)

model_mf = MF()
model_mf = model_mf(user_info, movie_info)
model_dnn_info = DNN()
model_dnn_info = model_dnn_info(user_info, movie_info, False)

for i in range(7):
    X_train = pd.read_csv(f'../data/interim/X_train_{i+1}.csv')
    X_test = pd.read_csv(f'../data/interim/X_test_{i+1}.csv')

    user_train = X_train['user_id']
    movie_train = X_train['item_id']
    ratings_train = X_train['ratings']

    user_test = X_test['user_id']
    movie_test = X_test['item_id']
    ratings_test = X_test['ratings']

    model_dir = f'../models/train_all/u{i+1}'
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    model_name_mf = os.path.join(model_dir, "mf_model_e{epoch:02d}.hdf5")
    model_name_dnn = os.path.join(model_dir, "dnn_model_e{epoch:02d}.hdf5")
    model_name_dnn_info = os.path.join(model_dir, "dnn_info_model_e{epoch:02d}.hdf5")
    
    train(model_mf, model_dnn_info, user_info, movie_info, user_train, movie_train, ratings_train, user_test, movie_test, ratings_test, model_name_mf, model_name_dnn, model_name_dnn_info)

Model: "model_32"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_82 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 input_83 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 embedding_192 (Embedding)      (None, 1, 16)        15104       ['input_82[0][0]']               
                                                                                                  
 embedding_195 (Embedding)      (None, 1, 16)        26928       ['input_83[0][0]']               
                                                                                           

Epoch 3/100
313/313 [==============================] - 1s 3ms/step - loss: 0.7894 - val_loss: 0.7825
Epoch 4/100
313/313 [==============================] - 1s 3ms/step - loss: 0.7128 - val_loss: 0.7327
Epoch 5/100
313/313 [==============================] - 1s 3ms/step - loss: 0.6492 - val_loss: 0.7018
Epoch 6/100
313/313 [==============================] - 1s 3ms/step - loss: 0.6005 - val_loss: 0.6843
Epoch 7/100
313/313 [==============================] - 1s 3ms/step - loss: 0.5623 - val_loss: 0.6752
Epoch 8/100
313/313 [==============================] - 1s 3ms/step - loss: 0.5313 - val_loss: 0.6716
Epoch 9/100
313/313 [==============================] - 1s 3ms/step - loss: 0.5053 - val_loss: 0.6718
Epoch 10/100
313/313 [==============================] - 1s 3ms/step - loss: 0.4830 - val_loss: 0.6741
Epoch 11/100
313/313 [==============================] - 1s 3ms/step - loss: 0.4638 - val_loss: 0.6784
Epoch 12/100
313/313 [==============================] - 1s 3ms/step - loss: 0.4469 - val_

Epoch 63/100
313/313 [==============================] - 4s 13ms/step - loss: 0.8452 - val_loss: 0.8875
Epoch 64/100
313/313 [==============================] - 4s 13ms/step - loss: 0.8441 - val_loss: 0.8876
Epoch 65/100
313/313 [==============================] - 4s 13ms/step - loss: 0.8429 - val_loss: 0.8879
Epoch 66/100
313/313 [==============================] - 4s 13ms/step - loss: 0.8425 - val_loss: 0.8899
Epoch 67/100
313/313 [==============================] - 4s 13ms/step - loss: 0.8414 - val_loss: 0.8865
Epoch 68/100
313/313 [==============================] - 4s 13ms/step - loss: 0.8418 - val_loss: 0.8962
Epoch 69/100
313/313 [==============================] - 4s 13ms/step - loss: 0.8406 - val_loss: 0.8864
Epoch 70/100
313/313 [==============================] - 4s 13ms/step - loss: 0.8401 - val_loss: 0.8891
Epoch 71/100
313/313 [==============================] - 4s 13ms/step - loss: 0.8407 - val_loss: 0.8871
Epoch 72/100
313/313 [==============================] - 4s 13ms/step - lo

 flatten_198 (Flatten)          (None, 64)           0           ['embedding_198[0][0]']          
                                                                                                  
 flatten_201 (Flatten)          (None, 64)           0           ['embedding_201[0][0]']          
                                                                                                  
 dropout_80 (Dropout)           (None, 64)           0           ['flatten_198[0][0]']            
                                                                                                  
 dropout_81 (Dropout)           (None, 64)           0           ['flatten_201[0][0]']            
                                                                                                  
 concatenate_16 (Concatenate)   (None, 128)          0           ['dropout_80[0][0]',             
                                                                  'dropout_81[0][0]']             
          

313/313 [==============================] - 4s 13ms/step - loss: 0.8160 - val_loss: 0.8434
Epoch 26/100
313/313 [==============================] - 4s 12ms/step - loss: 0.8183 - val_loss: 0.8437
Epoch 27/100
313/313 [==============================] - 4s 13ms/step - loss: 0.8167 - val_loss: 0.8450
Epoch 28/100
313/313 [==============================] - 4s 13ms/step - loss: 0.8169 - val_loss: 0.8470
Epoch 29/100
313/313 [==============================] - 4s 13ms/step - loss: 0.8152 - val_loss: 0.8473
Epoch 30/100
313/313 [==============================] - 4s 12ms/step - loss: 0.8154 - val_loss: 0.8476
Epoch 31/100
313/313 [==============================] - 4s 13ms/step - loss: 0.8153 - val_loss: 0.8482
Epoch 32/100
313/313 [==============================] - 4s 14ms/step - loss: 0.8148 - val_loss: 0.8471
Epoch 33/100
313/313 [==============================] - 5s 14ms/step - loss: 0.8136 - val_loss: 0.8494
Epoch 34/100
313/313 [==============================] - 4s 13ms/step - loss: 0.8143 - 

...

 concatenate_16 (Concatenate)   (None, 128)          0           ['dropout_80[0][0]',             
                                                                  'dropout_81[0][0]']             
                                                                                                  
 dense_64 (Dense)               (None, 256)          33024       ['concatenate_16[0][0]']         
                                                                                                  
 batch_normalization_48 (BatchN  (None, 256)         1024        ['dense_64[0][0]']               
 ormalization)                                                                                    
                                                                                                  
 dropout_82 (Dropout)           (None, 256)          0           ['batch_normalization_48[0][0]'] 
                                                                                                  
 dense_65 

354/354 [==============================] - 4s 12ms/step - loss: 0.7891 - val_loss: 0.7829
Epoch 34/100
354/354 [==============================] - 4s 12ms/step - loss: 0.7898 - val_loss: 0.7841
Epoch 35/100
354/354 [==============================] - 4s 12ms/step - loss: 0.7906 - val_loss: 0.7871
Epoch 36/100
354/354 [==============================] - 4s 12ms/step - loss: 0.7885 - val_loss: 0.7903
Epoch 37/100
354/354 [==============================] - 4s 12ms/step - loss: 0.7881 - val_loss: 0.7882
Epoch 38/100
354/354 [==============================] - 4s 12ms/step - loss: 0.7896 - val_loss: 0.7891
Epoch 39/100
354/354 [==============================] - 4s 12ms/step - loss: 0.7897 - val_loss: 0.7885
Epoch 40/100
354/354 [==============================] - 4s 12ms/step - loss: 0.7890 - val_loss: 0.7937
Epoch 41/100
354/354 [==============================] - 4s 12ms/step - loss: 0.7888 - val_loss: 0.7925
Epoch 42/100
354/354 [==============================] - 4s 12ms/step - loss: 0.7887 - 

                                                                                                  
 flatten_195 (Flatten)          (None, 16)           0           ['embedding_195[0][0]']          
                                                                                                  
 embedding_193 (Embedding)      (None, 1, 1)         944         ['input_82[0][0]']               
                                                                                                  
 embedding_196 (Embedding)      (None, 1, 1)         1683        ['input_83[0][0]']               
                                                                                                  
 dot_16 (Dot)                   (None, 1)            0           ['flatten_192[0][0]',            
                                                                  'flatten_195[0][0]']            
                                                                                                  
 flatten_1

Epoch 15/100
354/354 [==============================] - 1s 3ms/step - loss: 0.2737 - val_loss: 0.3824
Epoch 16/100
354/354 [==============================] - 1s 3ms/step - loss: 0.2727 - val_loss: 0.3937
Epoch 17/100
354/354 [==============================] - 1s 4ms/step - loss: 0.2718 - val_loss: 0.4045
Epoch 18/100
354/354 [==============================] - 1s 3ms/step - loss: 0.2709 - val_loss: 0.4152
Epoch 19/100
354/354 [==============================] - 1s 3ms/step - loss: 0.2701 - val_loss: 0.4255
Epoch 20/100
354/354 [==============================] - 1s 4ms/step - loss: 0.2694 - val_loss: 0.4359
Epoch 21/100
354/354 [==============================] - 1s 3ms/step - loss: 0.2688 - val_loss: 0.4455
Epoch 22/100
354/354 [==============================] - 1s 3ms/step - loss: 0.2682 - val_loss: 0.4561
Epoch 23/100
354/354 [==============================] - 1s 3ms/step - loss: 0.2677 - val_loss: 0.4660
Epoch 24/100
354/354 [==============================] - 1s 3ms/step - loss: 0.2671

Epoch 75/100
354/354 [==============================] - 4s 12ms/step - loss: 0.7834 - val_loss: 0.7990
Epoch 76/100
354/354 [==============================] - 4s 12ms/step - loss: 0.7850 - val_loss: 0.7963
Epoch 77/100
354/354 [==============================] - 4s 12ms/step - loss: 0.7834 - val_loss: 0.7996
Epoch 78/100
354/354 [==============================] - 4s 12ms/step - loss: 0.7826 - val_loss: 0.7983
Epoch 79/100
354/354 [==============================] - 5s 13ms/step - loss: 0.7833 - val_loss: 0.8009
Epoch 80/100
354/354 [==============================] - 5s 13ms/step - loss: 0.7829 - val_loss: 0.8044
Epoch 81/100
354/354 [==============================] - 4s 13ms/step - loss: 0.7832 - val_loss: 0.8011
Epoch 82/100
354/354 [==============================] - 4s 13ms/step - loss: 0.7838 - val_loss: 0.8033
Epoch 83/100
354/354 [==============================] - 5s 13ms/step - loss: 0.7845 - val_loss: 0.8080
Epoch 84/100
354/354 [==============================] - 5s 13ms/step - lo